# League link

In [5]:
import pandas as pd

import requests
from bs4 import BeautifulSoup
import numpy as np
def refreehist_extraction(leagues_list, today_date):
    leagues_dataset = {} #Created the empyt dictionary that will be used to concatenate all table from all leagues

    for key in list(leagues_list.keys()): #Loops through all the leagues in our list of league url
        league_url = leagues_list[key][1]

        try:
            if league_url == '': 
                #Checks for empty league links and skips
                continue
            else:
                #Gets the contents from the leagues schedule page
                response = requests.get(league_url)
                time.sleep(2)
                soup = BeautifulSoup(response.content, "html.parser")

                #Extracts the table with the match schedules
                table = soup.find("table", class_="standard_tabelle")

                data = []
                match_links = []

                #Extracts the table rows
                table_rows = table.find_all("tr")
                for row in table_rows:
                    #Loops through al the rows and extracts the data in each columns
                    columns = row.find_all("td")
                    row_data = [column.get_text(strip=True) for column in columns]
                    data.append(row_data)

                    #Extracts the urls of each match
                    match_link = row.find("a", href=lambda href: href and "report" in href)
                    if match_link:
                        match_links.append(match_link["href"])
                    else:
                        match_links.append('')

                #Drops the empty entries
                zipped_lists = zip(data, match_links)
                zipped_lists = [pair for pair in zipped_lists if len(pair[0]) > 0]
                data, match_links = zip(*zipped_lists)

                columns = ['Date', 'Time', 'Home Team', 'Score', 'Away Team', 'Result', 'Links']
                df = pd.DataFrame(data, columns=columns)
                df.Links = match_links

                df = df[df['Links'] != ''] #Drops columns with empty url

                #Add the prefix to the column
                df['Links'] = 'https://www.worldfootball.net' + df['Links']
                df['Date'] = df['Date'].replace('', np.nan).ffill()
                df['Date'] = pd.to_datetime(df['Date'],  format='%d/%m/%Y')
                today = date.today()

                #Filter rows with today's date
                today_df = df[df['Date'].dt.date == today_date] #Account for when the dataset filter everything due to no matching date
                today_df = today_df.copy(deep=True)
                curr_league = [key for i in range(len(today_df['Links']))]
                today_df['League'] = curr_league

                #Extracts the link to the profile of the officiating referee from the match page using match url
                referee_urls = []
                for match_url in today_df.Links:
                    response = requests.get(match_url)
                    html_content = response.content
                    soup = BeautifulSoup(html_content, "html.parser")
                    referee_links = soup.find_all("a", href=lambda href: href and "referee_summary" in href)
                    ref_link = []
                    if len(referee_links) > 0:
                        for link in referee_links:
                            link_url = link.get("href")
                            ref_link.append(f'https://www.worldfootball.net{link_url}')
                            
                        ref_link = ref_link[0]
                        referee_urls.append(ref_link)
                    else:
                        referee_urls.append('')

                #Add the url of the profile of the officiating referee of each match to dataframe containing daily matches
                today_df['Referee_Links'] = referee_urls
                #Extracts the url of the most recent matches officiated by the officiating referee
                ref_matchhist_url = []
                for ref_url in today_df.Referee_Links:
                    if ref_url != '':
                        ref_matchhist = []

                        response = requests.get(ref_url)
                        html_content = response.content

                        soup = BeautifulSoup(html_content, "html.parser")

                        table = soup.find("table")
                        rows = table.find_all("tr")

                        for row in rows:
                            columns = row.find_all("td")
                            for column in columns:
                                (column.get_text())

                        referee_summary_links = soup.find_all("a", href=lambda href: href and "referee_summary" and "2023-2024" and "2022-2023" in href)

                        for link in referee_summary_links:
                            link_url = link.get("href")
                            ref_matchhist.append(f'https://www.worldfootball.net{link_url}')
                        ref_matchhist = [link for link in ref_matchhist if 'referee_summary' in link]

                        ref_matchhist_url.append(ref_matchhist)
                    else:
                        ref_matchhist_url.append([])

                #Add the urls of the most recent matches officiated by the referee to the dataframe of daily matches
                today_df['Referee_MatchHist_Links'] = ref_matchhist_url

                #Extracts the details from each match and stores in a dictionary
                ref_matchhist_detail = []
                for row in today_df.Referee_MatchHist_Links:
                     if row != []:
                        data_dict = {'Date':[], 'Home Team':[], 'Away Team':[], 'Score':[], 'Yellow Cards':[], 'Unkown Card':[], 'Red Cards':[]}
                        for ref_match_url in row:
                            response = requests.get(ref_match_url)
                            soup = BeautifulSoup(response.content, "html.parser")

                            table = soup.find("table", class_="standard_tabelle")

                            data = [] # List to store table data

                            table_rows = table.find_all("tr")
                            for row in table_rows:
                                columns = row.find_all("td")
                                row_data = [column.get_text(strip=True) for column in columns]
                                data.append(row_data)

                            data = data[1:]
                            for entry in data:
                                entry.pop(2)

                            for entry in data:
                                for i in range(len(entry)):
                                    keys = list(data_dict.keys())[i]
                                    data_dict[keys].append(entry[i])
                        ref_matchhist_detail.append(data_dict)
                    else:
                        ref_matchhist_detail.append({})

                #Extracted match details are added to the dataframe of daily matches
                today_df['Referee_MatchHist_Details'] = ref_matchhist_detail

            leagues_dataset[key] = today_df #Adds the dataframe of daily matches for a league to the dictionary of leagues
        except:
            continue

    #All the dataframe of the daily matches for all the leagues extracted are concatenated vertically
    list_of_keys = list(leagues_dataset.keys())
    if len(list_of_keys) > 0:
        for i in range(len(list_of_keys)):
            if i == 0:
                key = list_of_keys[i]
                final_dataset = leagues_dataset[key].copy(deep=True)
            else:
                key = list_of_keys[i]
                final_dataset = pd.concat([final_dataset, leagues_dataset[key]], axis=0)

        return final_dataset
    else:
        print('No league scheduled matches were extracted!')
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 147)

# Match link

In [1]:
testleagues_list = {
    #'English Premier League':['https://www.flashscore.nl/voetbal/engeland/premier-league/schema/', 'https://www.worldfootball.net/all_matches/eng-premier-league-2023-2024/'],
    #'EFL Championship':['https://www.flashscore.nl/voetbal/engeland/efl-cup/schema/', ''],
    #'EFL Trophy':['https://www.flashscore.nl/voetbal/engeland/efl-trophy/schema/', 'https://www.worldfootball.net/all_matches/eng-efl-trophy-2023-2024/'],
    #'La Liga':['https://www.flashscore.nl/voetbal/spanje/laliga/schema/','https://www.worldfootball.net/all_matches/esp-primera-division-2023-2024/'],
    #'La Liga2':['https://www.flashscore.nl/voetbal/spanje/laliga2/schema/', 'https://www.worldfootball.net/all_matches/esp-segunda-division-2023-2024/']
    'Serie B':['https://www.flashscore.nl/voetbal/italie/serie-b/schema/', 'https://www.worldfootball.net/all_matches/ita-serie-b-2023-2024/'],
}
referee_dataset = refreehist_extraction(testleagues_list, tomorrow)
referee_dataset

NameError: name 'refreehist_extraction' is not defined

# Referee link

In [55]:
from bs4 import BeautifulSoup
import requests

url = "https://www.worldfootball.net//referee_summary/michael-salisbury/"

response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, "html.parser")

table = soup.find("table")
rows = table.find_all("tr")

for row in rows:
    columns = row.find_all("td")
    for column in columns:
        (column.get_text())


print("=" * 50) 
referee_summary_links = soup.find_all("a", href=lambda href: href and "referee_summary" and "2023-2024" and "2022-2023" in href)

# Extract and print the href attributes of the links
for link in referee_summary_links:
    link_url = link.get("href")
    print(link_url)


/referees/eng-premier-league-2022-2023/1/
/referee_summary/michael-salisbury/3/1/eng-premier-league-2022-2023/
/referees/eng-championship-2022-2023/1/
/referee_summary/michael-salisbury/3/1/eng-championship-2022-2023/
/referees/eng-fa-cup-2022-2023/1/
/referee_summary/michael-salisbury/3/1/eng-fa-cup-2022-2023/
/referees/eng-league-cup-2022-2023/1/
/referee_summary/michael-salisbury/3/1/eng-league-cup-2022-2023/
/referees/eng-playoffs-2022-2023-league-one/1/
/referee_summary/michael-salisbury/3/1/eng-playoffs-2022-2023-league-one/


# Refereed link

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.worldfootball.net/referee_summary/john-brooks_2/3/1/eng-championship-2022-2023/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", class_="standard_tabelle")

data = []  # List to store table data

table_rows = table.find_all("tr")
for row in table_rows:
    columns = row.find_all("td")
    row_data = [column.get_text(strip=True) for column in columns]
    data.append(row_data)

data = data[1:]

columns = ['Date', 'Home Team', 'Score', 'Away Team', 'Result', 'Links']
df = pd.DataFrame(data, columns=columns)


[['04/10/2022', 'Cardiff City', '-', 'Blackburn Rovers', '1:0', '4', '-', '-'],
 ['01/11/2022', 'Luton Town', '-', 'Reading FC', '0:0', '5', '-', '-'],
 ['26/12/2022', 'Hull City', '-', 'Blackpool FC', '1:1', '4', '-', '1'],
 ['25/02/2023', 'Reading FC', '-', 'Blackpool FC', '3:1', '1', '-', '-']]